In [1]:
import numpy as np
import pandas as pd

from trig import *

from scipy.optimize import fsolve
import math

import matplotlib.pyplot as plt

In [2]:
#var inputs

peak_torque_τ = 20
current_limit_i_ph = 115 #(a_rms)
voltage_limit = 48 #(v_dc)

In [3]:
#calculation

def calculate_max_no_load_rpm(peak_torque_τ,current_limit_i_ph,voltage_limit):
    kt = peak_torque_τ/current_limit_i_ph

    ke = kt/(3**0.5)*(1000*2*np.pi/60)

    max_no_load_rpm = voltage_limit/(2**0.5)/ke*1000
    
    return kt,ke,max_no_load_rpm

kt,ke,max_no_load_rpm = calculate_max_no_load_rpm(peak_torque_τ,current_limit_i_ph,voltage_limit)
kt,ke,max_no_load_rpm

(0.17391304347826086, 10.514778923096914, 3227.944757107419)

In [4]:
#var
#mean_airgap_radius r_g
r_ag = 41.75
trv = 0.10

In [5]:
#calc


stack_length = peak_torque_τ/trv*1000/(np.pi*r_ag**2)

σ = trv/2*(10**6)

σ, stack_length

(50000.0, 36.52306055391481)

In [6]:
#var
b_r = 1.28 # of magnet (T)

ag = 0.50 #Airgap length (mm)
l_pm = 5 #: Length of magnet (mm)
w_m = 20 #Width of magnet (mm)
k_pm = 0.95 #: Leakage flux factor
μ_ra = 1 #: Rel perm of magnet
b_sat = 1.5 #of steel (T)

no_of_poles = 10
no_of_slots = 12

assumed_slot_width_ratio = 0.75 #w' or w1


def calculate_slot_width_ratio(r,slot_pitch,assumed_slot_width_ratio,ag,k_pm,b_r,μ_ra,σ):
    
    slot_opening_width = slot_pitch*assumed_slot_width_ratio

    a = slot_opening_width/(2*ag)

    airgap_coeff = 4/np.pi*(a*np.arctan(a)-np.log(np.sqrt(1+a**2)))

    k_c = slot_pitch/(slot_pitch-airgap_coeff*ag)

    b_g = k_pm*b_r/(1+μ_ra*k_c*r)

    b_avg = (2/np.pi)*b_g

    K = (4/np.pi)*(σ/b_avg)/1000

    w = 1-b_avg/b_sat
    
    return [w,a,airgap_coeff,k_c,b_g,b_avg,K,slot_opening_width]


def solve_slot_width(assumed_slot_width_ratio,r,slot_pitch,ag,k_pm,b_r,μ_ra,σ):
    
    w = calculate_slot_width_ratio(r,slot_pitch,assumed_slot_width_ratio,ag,k_pm,b_r,μ_ra,σ)[0]
    
    return w/assumed_slot_width_ratio-1

def solve_number_of_turns(current_limit_i_ph,no_of_slots,i_total):

    number_of_turns = (i_total*1000)/(2*no_of_slots*current_limit_i_ph)
    
    return np.rint(number_of_turns)-number_of_turns

In [7]:
#calc

r = ag*w_m*no_of_poles/(2*np.pi*r_ag*l_pm)

slot_pitch = 2*np.pi*r_ag/no_of_slots

#to make w1 = w

solution = fsolve(solve_slot_width, assumed_slot_width_ratio, args=(r,slot_pitch,ag,k_pm,b_r,μ_ra,σ))
assumed_slot_width_ratio = solution[0]


w,a,airgap_coeff,k_c,b_g,b_avg,K,slot_opening_width = calculate_slot_width_ratio(r,slot_pitch,assumed_slot_width_ratio,ag,k_pm,b_r,μ_ra,σ)

i_total = round(K*2*np.pi*r_ag/1000)


#make no. of strands whole number by changing i_ph

solution = fsolve(solve_number_of_turns,current_limit_i_ph, args=(no_of_slots,i_total))
current_limit_i_ph = solution[0]

#update values after modifying a_rms
kt,ke,max_no_load_rpm = calculate_max_no_load_rpm(peak_torque_τ,current_limit_i_ph,voltage_limit)

number_of_turns = round((i_total*1000)/(2*no_of_slots*current_limit_i_ph))

tooth_thickness = round(2*np.pi*r_ag*(1-w)/no_of_slots)

yoke_thickness =round(1.2*(r_ag/no_of_poles))



In [8]:
r,slot_pitch,slot_opening_width,a,airgap_coeff,k_c,b_g,b_avg,K,w, i_total,number_of_turns,tooth_thickness,yoke_thickness

(0.07624188890629716,
 21.86024888122898,
 11.940778537158424,
 11.940778537158424,
 19.449250077254533,
 1.801328812161208,
 1.0691644682231736,
 0.6806512403837429,
 93.53097953787065,
 0.546232506410838,
 25,
 9,
 10,
 5)

In [9]:
#var

fill_factor = 41 #%
wire_dia = 0.914 #d_w
tooth_depth_factor = 0.36

def solve_number_of_strands(fill_factor,w,tooth_depth,b_avg,current_limit_i_ph,z,σ,wire_dia):
    
    number_of_strands = (fill_factor*w*tooth_depth*b_avg*current_limit_i_ph*z/(σ*(wire_dia**2))*1000)/100

    return np.rint(number_of_strands)-number_of_strands

In [10]:
#calc for radial

#for axial z =1 ignore for now

tooth_depth = tooth_depth_factor*r_ag

z = (tooth_depth+2*r_ag)/(2*r_ag)


#make number of strands whole number by changing fillfactor


solution = fsolve(solve_number_of_strands,fill_factor,args=(w,tooth_depth,b_avg,current_limit_i_ph,z,σ,wire_dia))
fill_factor = solution[0]

number_of_strands = round((fill_factor*w*tooth_depth*b_avg*current_limit_i_ph*z/(σ*(wire_dia**2))*1000)/100)




In [11]:
#var
resistivity_of_conductor = 1.72E-08 #_(ωm)
temp_coeff_of_resistivity_per_deg = 0.40 #_(/deg) % 
density_of_conductor = 9 #_(g/cc)
rs_per_g_of_conductor = 0.9 #_(g/cc)

grade_of_electric_steel = 250
steel_thickness = 0.35
density = 7.8 #_(g/cc)
rs_per_g_of_steel = 0.2
density_of_magnet = 7.8 #_(g/cc)
rs_per_g_of_magnet = 3


In [12]:
#var

cu_fe_HTC = 200 #[W/(m2K)]
fin_area_factor = 2.9
housing_by_stacklength_ratio = 3.5
heat_transfer_coeff = 31
ambient_temperature = 40 #(Deg-C)

#lambda in W/mK
lambda_steel = 28 #or 45
lambda_aluminium = 205

In [13]:
#calc for radial geometry

slot_depth = tooth_depth

rotor_od = 2*r_ag-ag

slot_od = rotor_od+2*ag+2*tooth_depth

stator_od = slot_od+yoke_thickness*2

rotor_id = rotor_od-2*yoke_thickness

mean_slot_dia = (rotor_od+2*ag+slot_od)/2


In [14]:
#calc for radial

z2 = mean_slot_dia/(2*r_ag)

l_ph = 2*number_of_turns*no_of_slots/3*(stack_length+slot_pitch)*z2

a_ph = 2*np.pi*(fill_factor/100)*w*tooth_depth*r_ag/(2*number_of_turns*no_of_slots)*z

#comeback
#r_ph = resistivity_of_conductor*(1+temp_coeff_of_resistivity_per_deg*(T24-25))*G30/G31*10^6*1.33

r_ph=38



In [15]:
#cost calculation radial

stator_steel_mass = (np.pi*(stator_od**2-slot_od**2)/4+tooth_thickness*slot_depth*no_of_slots)*stack_length/1000*density

stator_steel_cost = stator_steel_mass*rs_per_g_of_steel


magnet_mass = w_m*l_pm*no_of_poles*stack_length/1000*density_of_magnet

magnet_cost = magnet_mass*rs_per_g_of_magnet


rotor_steel_mass = (np.pi*(rotor_od**2-rotor_id**2)/4*stack_length-magnet_mass/density_of_magnet)*density/1000

rotor_steel_cost = rotor_steel_mass*rs_per_g_of_steel


conductor_mass = 3*l_ph*a_ph/1000*density_of_conductor

conductor_cost = conductor_mass*rs_per_g_of_conductor


total_mass = stator_steel_mass+rotor_steel_mass+magnet_mass+conductor_mass
total_cost = stator_steel_cost+rotor_steel_cost+magnet_cost+conductor_cost


In [17]:
assumed_winding_temp=158
r_ph = resistivity_of_conductor*(1+temp_coeff_of_resistivity_per_deg/100*(assumed_winding_temp-25))*l_ph/a_ph*(10**6)*1.33



In [21]:
#calc radial thermal resistance (tr)

#celsius/W

slot_perimeter = slot_pitch-tooth_thickness+2*slot_depth

conduction_area = slot_perimeter*stack_length*no_of_slots*(10**-6)

R_conduction = 1/cu_fe_HTC/conduction_area

convection_area = np.pi*stator_od*(stack_length*fin_area_factor*housing_by_stacklength_ratio+stator_od/2)*(10**-6)

R_convection = 1/(convection_area*heat_transfer_coeff)

print("here!!!!!!!!!!\nstack length: {}\nstator od: {}\nfin area factor: {}\nhousing by stack_length: {}\nconvection_area: {}\n".format(stack_length,stator_od,fin_area_factor,housing_by_stacklength_ratio,convection_area))


here!!!!!!!!!!
stack length: 36.52306055391481
stator od: 124.06
fin area factor: 2.9
housing by stack_length: 3.5
convection_area: 0.16865831281795843



In [26]:
#performance_analysis

#inputs

motor_speed_rpm = 2000
motor_torque = 8

#calc
electrical_frequency = motor_speed_rpm/60*no_of_poles/2

a_rms = motor_torque/kt

assumed_winding_temp = 158

In [27]:

def performance_analysis(r_ph,a_rms,electrical_frequency,no_of_slots,motor_speed_rpm,motor_torque,convection_area,R_convection,R_conduction,ambient_temperature):
    #peformance_analysis
    #Losses 

    copper_loss = 3*a_rms**2*r_ph/1000

    steel_loss = 26 #given for m350-50a

    total_loss = copper_loss+steel_loss

    #power kW

    output_power = motor_speed_rpm*motor_torque*2*np.pi/60/1000

    input_power = output_power+total_loss/1000

    efficiency = output_power/input_power


    #thermal steady state

    heat_flux = total_loss/convection_area*10**-4

    body_temperature = total_loss*R_convection+ambient_temperature

    winding_temperature = copper_loss*R_conduction+body_temperature

    
    cogging_frequency = electrical_frequency*no_of_slots
    
    return [winding_temperature,copper_loss,steel_loss,total_loss,output_power,input_power,efficiency,heat_flux,body_temperature,cogging_frequency]


In [28]:
def solve_winding_temperature(r_ph,assumed_winding_temp,a_rms,electrical_frequency,no_of_slots,motor_speed_rpm,motor_torque,convection_area,R_convection,R_conduction,ambient_temperature):
    
    winding_temperature = performance_analysis(r_ph,a_rms,electrical_frequency,no_of_slots,motor_speed_rpm,motor_torque,convection_area,R_convection,R_conduction,ambient_temperature)[0]
    
    return winding_temperature-assumed_winding_temp


print(assumed_winding_temp)

solution = fsolve(solve_winding_temperature,assumed_winding_temp,args=(assumed_winding_temp,a_rms,electrical_frequency,no_of_slots,motor_speed_rpm,motor_torque,convection_area,R_convection,R_conduction,ambient_temperature))
r_ph = solution[0]


winding_temperature,copper_loss,steel_loss,total_loss,output_power,input_power,efficiency,heat_flux,body_temperature,cogging_frequency = performance_analysis(r_ph,a_rms,electrical_frequency,no_of_slots,motor_speed_rpm,motor_torque,convection_area,R_convection,R_conduction,ambient_temperature)

print(winding_temperature,r_ph)


158
158.0 37.932133236622995


In [1]:
a ="peak_torque, current_limit_i_ph, voltage_limit, r_ag, trv, ag, no_of_poles, no_of_slots, fill_factor, wire_diameter, tooth_depth_factor, fin_area_factor, housing_by_stacklength_ratio, heat_transfer_coeff, ambient_temperature ,assumed_slot_width_ratio=0.75, assumed_winding_temp = 158"

In [2]:
for x in a.split(","):
    
    s = x.strip()
    print("self.{}".format(s),"=",s)

self.peak_torque = peak_torque
self.current_limit_i_ph = current_limit_i_ph
self.voltage_limit = voltage_limit
self.r_ag = r_ag
self.trv = trv
self.ag = ag
self.no_of_poles = no_of_poles
self.no_of_slots = no_of_slots
self.fill_factor = fill_factor
self.wire_diameter = wire_diameter
self.tooth_depth_factor = tooth_depth_factor
self.fin_area_factor = fin_area_factor
self.housing_by_stacklength_ratio = housing_by_stacklength_ratio
self.heat_transfer_coeff = heat_transfer_coeff
self.ambient_temperature = ambient_temperature
self.assumed_slot_width_ratio=0.75 = assumed_slot_width_ratio=0.75
self.assumed_winding_temp = 158 = assumed_winding_temp = 158


In [8]:
a=("a","b")

type(a)==tuple
a[0]

'a'

In [2]:
for x in '''Kt (Nm/A_rms)
Ke (V/Krpm)
Max no-load rpm
Stack length (L)
σ (N/m2)
r = g*W_m*P/[2π*r_g*L_pm]
Slot pitch p =  2π*r_g/S
Slot opg width W_s = p*w'
a = W_s/(2g)
Airgap coeff γ
Carter Coeff K_c = p/(p-γg)
Peak Airgap flux B_g 
B_avg =(2/π)*B_g
K (kA/m) = (4/π)*(σ/B_avg)
Actual slot-width ratio w
I_tot (KA) = K*(2πr_g)
Turns N = I_tot/(2S*I_ph)
Tooth thickness t 
Yoke thickness y'''.split("\n"):
    
    print("print('{}',)".format(x))

print('Kt (Nm/A_rms)',)
print('Ke (V/Krpm)',)
print('Max no-load rpm',)
print('Stack length (L)',)
print('σ (N/m2)',)
print('r = g*W_m*P/[2π*r_g*L_pm]',)
print('Slot pitch p =  2π*r_g/S',)
print('Slot opg width W_s = p*w'',)
print('a = W_s/(2g)',)
print('Airgap coeff γ',)
print('Carter Coeff K_c = p/(p-γg)',)
print('Peak Airgap flux B_g ',)
print('B_avg =(2/π)*B_g',)
print('K (kA/m) = (4/π)*(σ/B_avg)',)
print('Actual slot-width ratio w',)
print('I_tot (KA) = K*(2πr_g)',)
print('Turns N = I_tot/(2S*I_ph)',)
print('Tooth thickness t ',)
print('Yoke thickness y',)
